In [1]:
import fastbook
fastbook.setup_book()

In [2]:
from fastai.vision.all import *
from fastbook import *

In [3]:
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()
path = untar_data('https://s3.amazonaws.com/fast-ai-imageclas/mnist_png.tgz')

In [5]:
zeros, ones, twos, threes, fours, fives, sixs, sevens, eights, nines = [tensor(Image.open(o)) for o in (path/'training/0').ls()], [tensor(Image.open(o)) for o in (path/'training/1').ls()],[tensor(Image.open(o)) for o in (path/'training/2').ls()], [tensor(Image.open(o)) for o in (path/'training/3').ls()],[tensor(Image.open(o)) for o in (path/'training/4').ls()], [tensor(Image.open(o)) for o in (path/'training/5').ls()],[tensor(Image.open(o)) for o in (path/'training/6').ls()], [tensor(Image.open(o)) for o in (path/'training/7').ls()],[tensor(Image.open(o)) for o in (path/'training/8').ls()], [tensor(Image.open(o)) for o in (path/'training/9').ls()]


In [6]:
print(zeros)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
train_dset = list(zip(zeros,ones,twos,threes,fours,fives,sixs,sevens,eights,nines))

In [8]:
zeros, ones, twos, threes, fours, fives, sixs, sevens, eights, nines = [tensor(Image.open(o)) for o in (path/'testing/0').ls()], [tensor(Image.open(o)) for o in (path/'testing/1').ls()],[tensor(Image.open(o)) for o in (path/'testing/2').ls()], [tensor(Image.open(o)) for o in (path/'testing/3').ls()],[tensor(Image.open(o)) for o in (path/'testing/4').ls()], [tensor(Image.open(o)) for o in (path/'testing/5').ls()],[tensor(Image.open(o)) for o in (path/'testing/6').ls()], [tensor(Image.open(o)) for o in (path/'testing/7').ls()],[tensor(Image.open(o)) for o in (path/'testing/8').ls()], [tensor(Image.open(o)) for o in (path/'testing/9').ls()]


In [9]:
testing_dset = list(zip(zeros,ones,twos,threes,fours,fives,sixs,sevens,eights,nines))

In [12]:
training_dl = DataLoader(train_dset, batch_size=64)
testing_dl = DataLoader(testing_dset, batch_size=64)

In [14]:
dls = DataLoaders(training_dl, testing_dl)

In [15]:
simple_net = nn.Sequential(
    nn.Linear(28*28,30),
    nn.ReLU(),
    nn.Linear(30,1)
)

In [16]:
def batch_accuracy(xb, yb):
    preds = xb.sigmoid()
    correct = (preds>0.5) == yb
    return correct.float().mean()
def mnist_loss(predictions, targets):
    predictions = predictions.sigmoid()
    return torch.where(targets==1, 1-predictions, predictions).mean()

In [17]:
learn = Learner(dls, simple_net, opt_func=SGD,
                loss_func=mnist_loss, metrics=batch_accuracy)

** Do I need to overwrite forward for this?

In [18]:
learn.fit(10, .1)

TypeError: forward() takes 2 positional arguments but 10 were given